# Создание нейронной сети

В этом задании мы создадим полносвязную нейронную сеть используя при этом низкоуровневые механизмы tensorflow.

Архитектутра нейросети представлена на следующем рисунке. Как видите, в ней имеется один входной слой, два скрытых, а так же выходной слой. В качестве активационной функции в скрытых слоях будет использоваться сигмоида. На выходном слое мы используем softmax.

Часть кода по созданию сети уже написана, от вас требуется заполнить пропуски в указанных местах.

## Архитектура нейронной сети

<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg" alt="nn" style="width: 400px;"/>


## О датасете MNIST

Данную нейросеть мы будем обучать на датасете MNIST. Этот датасет представляет собой большое количество изображений рукописных цифр размером $28 \times 28$ пикселей. Каждый пиксель принимает значение от 0 до 255.

Как и раньше датасет будет разеделен на обучающую и тестовую выборки. При этом мы выполним нормализацию всех изображений, чтобы значения пикселей находились в промежутке от 0 до 1, разделив яркость каждого пикселя на 255.

Кроме того, архитектура нейронной сети ожидает на вход вектор. В нашем же случае каждый объект выборки представляет собой матрицу. Что же делать? В этом задании мы "растянем" матрицу $28 \times 28$, получив при этом вектор, состоящей из 784 элементов.

![MNIST Dataset](https://www.researchgate.net/profile/Steven-Young-5/publication/306056875/figure/fig1/AS:393921575309346@1470929630835/Example-images-from-the-MNIST-dataset.png)

Больше информации о датасете можно найти [здесь](http://yann.lecun.com/exdb/mnist/).

In [1]:
from typing import Optional, Union


import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tf.keras.activations import sigmoid
from tensorflow.keras.datasets import mnist
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.utils import to_categorical

#### O/S/E

In [8]:
num_classes = 10  # общее количество классов, в нашем случае это цифры от 0 до 9
num_features = 784  # количество атрибутов входного вектора 28 * 28 = 784

learning_rate = 0.001  # скорость обучения нейронной сети
training_steps = 3000  # максимальное число эпох
batch_size = 256  # пересчитывать веса сети мы будем не на всей выборке, а на ее случайном подможестве из batch_size элементов
display_step = 100  # каждые 100 итераций мы будем показывать текущее значение функции потерь и точности

n_hidden_1 = 128  # количество нейронов 1-го слоя
n_hidden_2 = 256  # количество нейронов 2-го слоя

buffer_size = 5000  # This dataset fills a buffer with buffer_size elements, then randomly samples elements from this buffer, replacing the selected elements with new elements.

- https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data
- https://www.tensorflow.org/api_docs/python/tf/data/Dataset
- https://www.tensorflow.org/api_docs/python/tf/data/Dataset#repeat
- https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle
- https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch
- https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch

In [3]:
# Загружаем датасет
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Преобразуем целочисленные пиксели к типу float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Преобразуем матрицы размером 28x28 пикселей в вектор из 784 элементов
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

# Нормализуем значения пикселей
x_train, x_test = x_train / 255., x_test / 255.

In [4]:
# Перемешаем тренировочные данные (split to batches & ...)
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(buffer_size).batch(batch_size).prefetch(1)

In [5]:
x_train.shape, x_test.shape

((60000, 784), (10000, 784))

In [6]:
y_train.shape, y_test.shape

((60000,), (10000,))

In [7]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 784), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

#### M

- https://www.tensorflow.org/api_docs/python/tf/keras/activations/sigmoid
- https://www.tensorflow.org/api_docs/python/tf/random/normal
- https://stackoverflow.com/questions/66968102/python-type-hint-can-tensorflow-data-type-be-used
- https://www.tensorflow.org/api_docs/python/tf/nn/relu
- https://www.tensorflow.org/api_docs/python/tf/nn/softmax

In [ ]:
# Создадим нейронную сеть

class DenseLayer(tf.Module):
    def __init__(self, in_features: int, out_features: int, name: Optional[str]=None) -> None:
        super().__init__(name=name)
        self.w = tf.Variable(
                             tf.random.normal([in_features, out_features]), 
                             name='w'
                             )
        # self.b = tf.Variable(tf.zeros([out_features]), name='b')  # b -> 0
        self.b = tf.Variable(tf.random.normal([out_features]), name='b')

    def __call__(self, x: tf.float32) -> tf.float32:
        y = tf.matmul(x, self.w) + self.b
        if self.name == 'softmax':
           
           return tf.nn.softmax(y)
        
        return sigmoid(y)


class NN(tf.Module):
  def __init__(self, name: Optional[str]=None) -> None:
    super().__init__(name=name)
    # Первый слой, состоящий из 128 нейронов
    self.layer_1 = DenseLayer(in_features=num_features, out_features=n_hidden_1)

    # Второй слой, состоящий из 256 нейронов
    self.layer_2 = DenseLayer(in_features=n_hidden_1, out_features=n_hidden_2)

    # Выходной слой
    self.layer_out = DenseLayer(in_features=n_hidden_2, out_features=num_classes, name='softmax')

  def __call__(self, x: tf.float32) -> tf.float32:
    x = self.layer_1(x)

    x = self.layer_2(x)

    # Помните что для выхода нейронной сети мы применяем к выходу функцию softmax. 
    # Делаем мы это для того, чтобы
    # выход нейронной сети принимал значения от 0 до 1 в соответствии с вероятностью 
    # принадлежности входного объекта к одному из 10 классов

    x = self.layer_out(x)
    
    return x  # tf.nn.softmax(x)

- https://www.tensorflow.org/api_docs/python/tf/one_hot
- https://www.tensorflow.org/api_docs/python/tf/clip_by_value
- https://www.tensorflow.org/api_docs/python/tf/math/reduce_mean
- https://uk.wikipedia.org/wiki/%D0%9F%D0%B5%D1%80%D0%B5%D1%85%D1%80%D0%B5%D1%81%D0%BD%D0%B0_%D0%B5%D0%BD%D1%82%D1%80%D0%BE%D0%BF%D1%96%D1%8F
- https://www.tensorflow.org/api_docs/python/tf/compat/v1/metrics/accuracy
- https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Accuracy

- https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy
- https://www.tensorflow.org/api_docs/python/tf/keras/metrics/categorical_crossentropy

In [ ]:
# В качестве функции ошибки в данном случае удобно взять кросс-энтропию (num_classes > 2)
def cross_entropy(y_pred: tf.float32, y_true: np.array) -> tf.float32:
    # Encode label to a one hot vector.
    # y_true = to_categorical(y_true, num_classes=num_classes)
    y_true = tf.one_hot(y_true, depth=num_classes)
    
    # Clip prediction values to avoid log(0) error. (acording to min and max value)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

    # Вычисление кросс-энтропии (reduce_mean - Computes the mean of elements across dimensions of a tensor.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))


# В качестве метрики качества используем точность
def accuracy(y_pred: tf.float32, y_true: np.array) -> float:
    m = Accuracy()
    m.update_state(y_true, y_pred)

    return m.result().numpy()
    # return accuracy_score(y_true, y_pred)

- https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/SGD
- https://www.tensorflow.org/api_docs/python/tf/GradientTape

In [ ]:
# num_classes = 10 # общее количество классов, в нашем случае это цифры от 0 до 9
# num_features = 784 # количество атрибутов входного вектора 28 * 28 = 784

# learning_rate = 0.001 # скорость обучения нейронной сети
# training_steps = 3000 # максимальное число эпох
# batch_size = 256 # пересчитывать веса сети мы будем не на всей выборке, а на ее случайном подможестве из batch_size элементов
# display_step = 100 # каждые 100 итераций мы будем показывать текущее значение функции потерь и точности

# n_hidden_1 = 128 # количество нейронов 1-го слоя
# n_hidden_2 = 256 # количество нейронов 2-го слоя

In [ ]:
# Создадим экзампляр нейронной сети
neural_net = NN(name="mnist")

# Функция обучения нейросети
def train(nn, input_x, output_y):
    # Для подгонки весов сети будем использовать стохастический градиентный спуск
    optimizer = tf.optimizers.SGD(learning_rate)  # .Adam

    # Активация автоматического дифференцирования
    with tf.GradientTape() as g:
      pred = neural_net(input_x)  # ? nn(input_x)
      loss = cross_entropy(pred, output_y)

      # Создадим оптимизируемых список параметров
      # Место для вашего кода
      params = [nn.w, nn.b]

      # Вычислим по ним значение градиента
      # Место для вашего кода
      dw, db = g.gradient(loss, params)

      
      # Модифицируем параметры
      # Место для вашего кода
      nn.w.assign_sub(learning_rate * dw)
      nn.b.assign_sub(learning_rate * db)

In [ ]:
# Тренировка сети

loss_history = []  # каждые display_step шагов сохраняйте в этом список текущую ошибку нейросети
accuracy_history = [] # каждые display_step шагов сохраняйте в этом список текущую точность нейросети

# В этом цикле мы будем производить обучение нейронной сети
# из тренировочного датасета train_data извлеките случайное подмножество, на котором 
# произведется тренировка. Используйте метод take, доступный для тренировочного датасета.
for step, (batch_x, batch_y) in train_data:  # Место для вашего кода:
    # Обновляем веса нейронной сети
    # Место для вашего кода
    train(neural_net, batch_x, batch_y, learning_rate=learning_rate)
    pred = neural_net(batch_x)
    current_loss = cross_entropy(batch_y, pred)
    print(f'loss: {current_loss}')  ##-
    
    if step % display_step == 0:
        # pred = neural_net(batch_x)

        # Место для вашего кода
        loss_history.append(current_loss)
        accuracy_history.append(accuracy(batch_y, pred))

In [ ]:
# Выведите графики зависимости изменения точности и потерь от шага
# Если все сделано правильно, то точность должна расти, а потери уменьшаться

# Место для вашего кода


In [ ]:
# Вычислите точность обученной нейросети

# Место для вашего кода

In [ ]:
# Протестируем обученную нейросеть на 10 изображениях. Из тестовой выборки возьмите 5 
# случайных изображений и передайте их в нейронню сеть.
# Выведите изображение и выпишите рядом ответ нейросети.
# Сделайте вывод о том ошибается ли ваша нейронная сеть и если да, то как часто?

# Место для вашего кода